In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install datasets

%cd /content/drive/Othercomputers/My MacBook Air/Thesis/Coding/Synth

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
/content/drive/Othercomputers/My MacBook Air/Thesis/Coding/Synth


In [2]:
!ls

 all_entries_with_labels.pkl		 logs
 augmenter.ipynb			 old_PharmBERT_runner_final.ipynb
 biomedical_ner_all_runner_final.ipynb	 output
 concatenated.json			 PharmBERT_runner_newwave.ipynb
 concatenated.txt			 pickle_opener.ipynb
 data_maker.ipynb			 reason_reader.ipynb
 example				 results
 fine_tuned_biobert			 runner.ipynb
 finetuned_biodemoical_ner_model	 run_tester.ipynb
 finetuned_pharmbert_model		 split_finetuned_pharmbert_model
 ground_truth_labeller.ipynb		 test
 input					 text_concaternator.ipynb
 labeled_records			 tokenized_data.pkl
'labeller copy.ipynb'			 tokenized_input_txt_maker.ipynb
 labeller.ipynb				 tokenizer.ipynb
 langchain.ipynb			 unique_values.json


In [1]:
# General imports
import os
import re
import json
import random
import string
import numpy as np
from pathlib import Path
from typing import List

# PyTorch imports
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss

# Hugging Face Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
    PreTrainedTokenizerBase
)

# Hugging Face Datasets
from datasets import Dataset

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report


/Users/remon.m/miniconda3/envs/PharmBERT/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# # Define input and output directories - COLAB
input_dir = '/content/drive/Othercomputers/My MacBook Air/Thesis/Coding/Synth/input' #change for test/ actual
output_dir = '/content/drive/Othercomputers/My MacBook Air/Thesis/Coding/Synth/output' #change for test/ actual
unique_values_json = '/content/drive/Othercomputers/My MacBook Air/Thesis/Coding/Synth/unique_values.json'

In [2]:
# Define input and output directories - LOCAL
# input_dir = '/Users/remon.m/Desktop/Y4S2/Thesis/Coding/Synth/input' #change for test/ actual
# output_dir = '/Users/remon.m/Desktop/Y4S2/Thesis/Coding/Synth/output' #change for test/ actual
# unique_values_json = '/Users/remon.m/Desktop/Y4S2/Thesis/Coding/Synth/unique_values.json'

In [3]:
# Load unique values from JSON
with open(unique_values_json, 'r', encoding='utf-8') as json_file:
    unique_values = json.load(json_file)

reasons_list = [reason.lower().strip() for reason in unique_values['reasons']]
drugs_list = [drug.lower().strip() for drug in unique_values['drug_names']]
forms_list = [form.lower().strip() for form in unique_values['forms']]
units_list = [unit.lower().strip() for unit in unique_values['units']]
dosages_list = [dosage.lower().strip() for dosage in unique_values['dosages']]
frequencies_list = [freq.lower().strip() for freq in unique_values['frequencies']]
routes_list = [route.lower().strip() for route in unique_values['routes']]

# Define patterns for different fields
reason_pattern = re.compile(
    r"\b(" + "|".join(re.escape(reason) for reason in reasons_list) + r")\b",
    re.IGNORECASE
)

drug_pattern = re.compile(
    r"\b(" + "|".join(re.escape(drug) for drug in drugs_list) + r")\b",
    re.IGNORECASE
)

unit_pattern = re.compile(
    r"\b(" + "|".join(["mg", "mcg", "g", "ml", "%", "unit", "drop", "dose"] +
    [re.escape(u) for u in units_list]) + r")\b",
    re.IGNORECASE
)

form_pattern = re.compile(
    r"\b(" + "|".join(re.escape(form) for form in forms_list) + r")\b",
    re.IGNORECASE
)

dosage_pattern = re.compile(
    r"\b(" + "|".join(["\d+", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"] +
    [re.escape(d) for d in dosages_list]) + r")\b",
    re.IGNORECASE
)

frequency_pattern = re.compile(
    r"\b(" + "|".join(re.escape(freq) for freq in frequencies_list) + r")\b",
    re.IGNORECASE
)

route_pattern = re.compile(
    r"\b(" + "|".join(re.escape(route) for route in routes_list) + r")\b",
    re.IGNORECASE
)

# Define label list
label_list = ["O", "B-DRUG", "I-DRUG", "B-STRENGTH", "I-STRENGTH",
              "B-FORM", "I-FORM", "B-DOSAGE", "I-DOSAGE",
              "B-FREQUENCY", "I-FREQUENCY", "B-ROUTE", "I-ROUTE",
              "B-REASON", "I-REASON"]

# Define the valid fields used for structured data extraction
valid_fields = ["DRUG", "STRENGTH", "FORM", "DOSAGE", "FREQUENCY", "ROUTE", "REASON"]

label_map = {
    "O": 0,
    "B-DRUG": 1,
    "I-DRUG": 2,
    "B-STRENGTH": 3,
    "I-STRENGTH": 4,
    "B-FORM": 5,
    "I-FORM": 6,
    "B-DOSAGE": 7,
    "I-DOSAGE": 8,
    "B-FREQUENCY": 9,
    "I-FREQUENCY": 10,
    "B-ROUTE": 11,
    "I-ROUTE": 12,
    "B-REASON": 13,
    "I-REASON": 14,
}

In [4]:
def load_raw_instructions(input_dir):
    """Load raw instructions from input files."""
    input_data = {}
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.txt'):
            file_path = os.path.join(input_dir, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                base_name = file_name.replace('.txt', '')
                input_data[base_name] = f.read().strip()
    return input_data

def load_structured_data(output_dir):
    """Load and format structured output data."""
    output_data = {}
    for file_name in os.listdir(output_dir):
        if file_name.endswith('.txt'):
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().split('Labels:')[0].strip()
                if content.startswith('Extracted Fields:'):
                    content = content[len('Extracted Fields:'):].strip()
                formatted_content = re.sub(r'(\d+\.)\s+', r'\n\1 ', content)
                base_name = file_name.replace('_redone_response.txt', '')
                output_data[base_name] = formatted_content.strip()
    return output_data

raw_instructions = load_raw_instructions(input_dir)
structured_data = load_structured_data(output_dir)

In [5]:
def align_data(input_data, output_data):
    """Align input and structured output data into a dataset."""
    dataset = []
    not_in = []

    for base_name, input_text in input_data.items():
        structured_text = output_data.get(base_name)

        if not structured_text:
            print(f"Warning: No matching output data found for {base_name}. Skipping.")
            not_in.append(base_name)
            continue

        # Split input and structured lines properly
        input_lines = [line.strip() for line in input_text.split('\n') if line.strip()]
        structured_lines = [line.strip() for line in structured_text.split('\n')
                            if line.strip() and "Extract" not in line and "```" not in line and "--" not in line]

        # Check if input lines match the number of structured lines
        if len(input_lines) != len(structured_lines):
            print(f"Warning: Mismatch in number of lines for {base_name}. Skipping.")
            print(len(input_lines), len(structured_lines))
            not_in.append(base_name)
            continue

        # Create entries for each input-output pair
        for input_line, structured_line in zip(input_lines, structured_lines):
            structured_dict = {}

            # Only process structured lines that match the expected format
            if re.match(r'^\d+\.\s*', structured_line) and " | " in structured_line:
                fields = [field.split(": ")[1] if ": " in field else "N/A"
                          for field in structured_line.split(" | ")]

                # Ensure fields contains enough elements before accessing them
                if len(fields) < len(valid_fields):
                    print(input_line, structured_line)
                    print(f"Warning: Mismatch in the number of fields for {base_name}. Skipping.")
                    print(f"Expected {len(valid_fields)} fields but got {len(fields)}.")
                    not_in.append(base_name)
                    continue

                drug_name = fields[0]

                if drug_name.lower() != "n/a":
                    structured_dict[drug_name] = {
                        valid_field: fields[idx] if idx < len(fields) else "N/A"
                        for idx, valid_field in enumerate(valid_fields)
                    }

                # Append the new sample to the dataset
                if structured_dict and input_line.lower() != "expired":
                    dataset.append({"instructions": input_line, "medications": structured_dict})

    return dataset, not_in

aligned_data, not_in = align_data(raw_instructions, structured_data)

In [6]:
# Initialize a set to collect unique words from the data
new_tokens_set = set()

# Simple space-based tokenizer function for vocabulary extraction
def simple_tokenizer(text: str):
    return text.split()

# Iterate through each entry in the aligned_data
for entry in aligned_data:
    # Extract the instructions and split into words
    instruction_tokens = simple_tokenizer(entry['instructions'])
    new_tokens_set.update(instruction_tokens)  # Add tokens to the set

    # Extract medication details
    medications = entry['medications']

    # Iterate through each medication in the entry
    for medication_name, fields in medications.items():
        # Add the medication name itself
        new_tokens_set.add(medication_name)

        # Iterate through each field in the medication dictionary
        for field_name, field_value in fields.items():
            # Split the field value into tokens and add them
            field_tokens = simple_tokenizer(field_value)
            new_tokens_set.update(field_tokens)

# Convert the set to a sorted list to maintain a consistent order
new_tokens = sorted(list(new_tokens_set))

class SimpleTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.token_to_id = {token: idx + 1 for idx, token in enumerate(vocab)}  # ID starts at 1
        self.unknown_token_id = len(self.token_to_id) + 1  # ID for unknown tokens

    def tokenize(self, text: str) -> List[str]:
        # Use the same splitting approach as simple_tokenizer
        return text.split()

    def convert_tokens_to_ids(self, tokens: List[str]) -> List[int]:
        # Handle unknown tokens consistently
        return [self.token_to_id.get(token, self.unknown_token_id) for token in tokens]

    def __call__(self, text: str):
        tokens = self.tokenize(text)
        input_ids = self.convert_tokens_to_ids(tokens)
        return {
            "input_ids": input_ids,
            "attention_mask": [1] * len(input_ids)  # Default attention mask for each token
        }

    def save_pretrained(self, save_directory):
        """Save the tokenizer vocabulary to a directory."""
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        vocab_file = os.path.join(save_directory, 'vocab.json')
        with open(vocab_file, 'w') as f:
            json.dump(self.token_to_id, f)

        print(f"Tokenizer vocabulary saved to {vocab_file}")

# Initialize the tokenizer with the new vocabulary
tokenizer = SimpleTokenizer(vocab=new_tokens)

In [7]:
def get_label_for_word(word, medications, previous_word=None, next_word=None, current_reason=None, reason_tracker=None):
    """Retrieve the correct label for a word based on context and structured data."""
    normalized_word = word.lower()

    if re.match(r"^\d+\.$", normalized_word.strip()) or normalized_word.strip() == '.':
        return "O", reason_tracker  # Not a dosage

    # Track multi-word reasons
    if reason_tracker:
        expected_word = reason_tracker["expected_word"]
        full_reason = reason_tracker["full_reason"]
        current_word_index = reason_tracker["index"]

        if normalized_word.strip(string.punctuation) == expected_word:  # Matches the next word in the multi-word reason
            # If it's the last word in the multi-word reason
            if current_word_index == len(full_reason) - 1:
                reason_tracker = None  # Reset the tracker after last word
            else:
                reason_tracker["index"] += 1  # Move to the next word in the multi-word reason
                reason_tracker["expected_word"] = full_reason[current_word_index]  # Update next word
            return "I-REASON", reason_tracker  # Continue tagging as I-REASON

    # Iterate through reasons to find multi-word reasons
    for reason in reasons_list:
        reason_words = reason.split()  # Split multi-word reason into individual words
        if normalized_word == reason_words[0]:  # First word of the reason
            if len(reason_words) > 1:  # It's a multi-word reason
                reason_tracker = {
                    "index": 1,  # Index of the next word to expect
                    "full_reason": reason_words,  # The full reason broken into words
                    "expected_word": reason_words[1],  # The next word to look for
                }
            return "B-REASON", reason_tracker  # Start of a multi-word reason

    # Existing checks for single-word reasons
    if re.match(reason_pattern, normalized_word):
        return ("B-REASON", reason_tracker) if current_reason != normalized_word else ("I-REASON", reason_tracker)

    # Handle strength detection (number followed by a unit)
    if re.match(dosage_pattern, normalized_word) and next_word and re.match(unit_pattern, next_word.lower()):
        return "B-STRENGTH", reason_tracker

    # Handle I-STRENGTH (continuation with a unit)
    if re.match(unit_pattern, normalized_word):
        return "I-STRENGTH", reason_tracker

    # Handle B-DOSAGE (number followed by a form)
    if re.match(dosage_pattern, normalized_word) and next_word and re.match(form_pattern, next_word.lower()):
        return "B-DOSAGE", reason_tracker

    # Handle dosage detection based on the previous word (number + unit)
    if previous_word and re.match(dosage_pattern, previous_word) and re.match(unit_pattern, normalized_word):
        return "I-DOSAGE", reason_tracker

    # Check if the current word is a dosage value (standalone number)
    if re.match(dosage_pattern, normalized_word):
        return ("B-DOSAGE", reason_tracker) if previous_word not in [".", ","] else ("O", reason_tracker)  # Avoid labeling numbered lists

    # Iterate through medications to find a match
    for drug, attributes in medications.items():
        if drug.lower() in normalized_word:
            return "B-DRUG", reason_tracker

        # Check for other attributes, including reason
        for field, value in attributes.items():
            if field.upper() == "REASON" and value.lower() in normalized_word:
                return ("B-REASON", reason_tracker) if current_reason != value.lower() else ("I-REASON", reason_tracker)
            if value.lower() in normalized_word:
                return f"B-{field.upper()}", reason_tracker

    return "O", reason_tracker  # Default to 'O' if no match is found

def tokenize_instructions(raw_instructions):
    """Tokenize raw instructions using a simple space-based tokenizer."""
    tokenized_data = []

    for base_name, instruction in raw_instructions.items():
        # Tokenize the raw instruction
        tokens = tokenizer(instruction)

        # Prepare entry for saving
        entry = {
            "raw_instruction": instruction,
            "tokenized_instruction": tokens['input_ids']  # Use input_ids instead of the whole output
        }

        tokenized_data.append(entry)

    return tokenized_data

def tokenize_and_align_for_dict(entry):
    """Tokenize input text and align labels with tokens using a simple tokenizer."""
    instructions = entry['instructions']
    medications = entry['medications']

    # Tokenize the input text using the simple space-based tokenizer
    tokens = tokenizer(instructions)  # This returns a dictionary with "input_ids"

    # Get the token strings from the input_ids
    token_strings = [tokenizer.vocab[token_id - 1] for token_id in tokens['input_ids']]  # Convert to strings

    aligned_labels = []
    current_reason = None  # Track the current reason for multi-word labeling
    reason_tracker = None  # Track the multi-word reason state

    for idx, token in enumerate(token_strings):
        previous_word = token_strings[idx - 1] if idx > 0 else None
        next_word = token_strings[idx + 1] if idx < len(token_strings) - 1 else None

        label, reason_tracker = get_label_for_word(
            token, medications,
            previous_word=previous_word,
            next_word=next_word,
            current_reason=current_reason,
            reason_tracker=reason_tracker
        )

        aligned_labels.append(label_map[label])  # Ensure label_map is defined

    return {"tokens": token_strings, "labels": aligned_labels}

# Example usage on the first entry of your aligned data
aligned_tokenized_data = [tokenize_and_align_for_dict(entry) for entry in aligned_data]

In [9]:
# Iterate through the first 3 entries in aligned_tokenized_data
for i in range(3):  # random 3 entries from the middle 27,30
    print(f"\n--- Entry {i + 1} ---")

    # Get the entry and extract tokens and labels
    entry = aligned_tokenized_data[i]
    tokens = entry["tokens"]  # Access tokens directly
    labels = entry["labels"]

    # Print the original input text for reference
    print("\nOriginal Input Text:")
    print(aligned_data[i]['instructions'])

    print("\nTokenized Tokens and Corresponding Labels:\n")

    # Display tokens with their aligned labels
    for token, label_id in zip(tokens, labels):
        label = label_list[label_id] if label_id != -100 else "PAD/SPECIAL"
        print(f"{token} : {label}")

print("\nFinished displaying the first 3 entries.")



--- Entry 1 ---

Original Input Text:
1. Duloxetine 60 mg Capsule Sig: One (1) Capsule PO DAILY for anxiety.

Tokenized Tokens and Corresponding Labels:

1. : O
Duloxetine : B-DRUG
60 : B-STRENGTH
mg : I-STRENGTH
Capsule : B-FORM
Sig: : O
One : B-DOSAGE
(1) : B-DOSAGE
Capsule : B-FORM
PO : B-ROUTE
DAILY : B-FREQUENCY
for : O
anxiety. : B-REASON

--- Entry 2 ---

Original Input Text:
2. Levothyroxine 75 mcg Tablet Sig: One (1) Tablet PO DAILY for hypothyroidism.

Tokenized Tokens and Corresponding Labels:

2. : O
Levothyroxine : B-DRUG
75 : B-STRENGTH
mcg : I-STRENGTH
Tablet : B-FORM
Sig: : O
One : B-DOSAGE
(1) : B-DOSAGE
Tablet : B-FORM
PO : B-ROUTE
DAILY : B-FREQUENCY
for : O
hypothyroidism. : B-REASON

--- Entry 3 ---

Original Input Text:
3. Loratadine 10 mg Tablet Sig: One (1) Tablet PO DAILY for allergies.

Tokenized Tokens and Corresponding Labels:

3. : O
Loratadine : B-DRUG
10 : B-STRENGTH
mg : I-STRENGTH
Tablet : B-FORM
Sig: : O
One : B-DOSAGE
(1) : B-DOSAGE
Tablet : B-FORM
P

In [10]:
# Example of converting the aligned_data using the SimpleTokenizer
updated_data = []

for idx, entry in enumerate(aligned_tokenized_data):
    if "tokens" not in entry:
        print(f"Warning: 'tokens' key missing in entry {idx}. Entry content: {entry}")
        continue

    # Tokenize the text and convert tokens to input IDs using the SimpleTokenizer
    tokenized_output = tokenizer(' '.join(entry["tokens"]))  # Join tokens with a space and tokenize
    input_ids = tokenized_output["input_ids"]
    attention_mask = tokenized_output["attention_mask"]

    # Create the updated entry with input_ids, attention_mask, and labels
    updated_entry = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": entry["labels"]
    }
    updated_data.append(updated_entry)

import pickle

with open('final_data.pkl', 'wb') as file:
    pickle.dump(updated_data, file)



In [15]:
updated_data

with open('aligned_tokenized_data.pkl', 'wb') as file:
    pickle.dump(aligned_tokenized_data, file)

with open('raw_data.pkl', 'wb') as file:
    pickle.dump(aligned_data, file)


In [191]:
# Check a sample entry to ensure correct conversion
for idx, (original_entry, updated_entry) in enumerate(zip(aligned_tokenized_data[:3], updated_data[:3])):
    print(f"Original Entry {idx + 1}:")
    print(f"Original Tokens: {original_entry['tokens']}")
    print(f"Labels: {original_entry['labels']}\n")

    print(f"Updated Entry {idx + 1}:")
    print(f"Input IDs: {updated_entry['input_ids']}")
    print(f"Tokens: {[new_tokens[i - 1] if i <= len(new_tokens) else '[UNK]' for i in updated_entry['input_ids']]}")
    print(f"Labels: {updated_entry['labels']}")
    print(f"Attention Mask: {updated_entry['attention_mask']}\n")
    print("-" * 50)

Original Entry 1:
Original Tokens: ['1.', 'Duloxetine', '60', 'mg', 'Capsule', 'Sig:', 'One', '(1)', 'Capsule', 'PO', 'DAILY', 'for', 'depression.']
Labels: [0, 1, 3, 4, 5, 0, 7, 7, 5, 11, 9, 0, 13]

Updated Entry 1:
Input IDs: [59, 470, 153, 1653, 339, 1068, 897, 1, 339, 912, 413, 1511, 1415]
Tokens: ['1.', 'Duloxetine', '60', 'mg', 'Capsule', 'Sig:', 'One', '(1)', 'Capsule', 'PO', 'DAILY', 'for', 'depression.']
Labels: [0, 1, 3, 4, 5, 0, 7, 7, 5, 11, 9, 0, 13]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

--------------------------------------------------
Original Entry 2:
Original Tokens: ['2.', 'Triptorelin', '3.75', 'mg', 'Injection', 'Sig:', 'One', '(1)', 'Injection', 'Intramuscular', 'Monthly', 'for', 'prostate', 'cancer.']
Labels: [0, 1, 3, 4, 5, 0, 7, 7, 5, 11, 9, 0, 13, 14]

Updated Entry 2:
Input IDs: [98, 1194, 119, 1653, 674, 1068, 897, 1, 674, 686, 823, 1511, 1800, 1353]
Tokens: ['2.', 'Triptorelin', '3.75', 'mg', 'Injection', 'Sig:', 'One', '(1)', 'Injection',

In [12]:
# Test tokenizer on sample tokens
sample_text = ' '.join(aligned_tokenized_data[0]["tokens"])
tokenized_output = tokenizer(sample_text)

print(f"Original Tokens: {aligned_tokenized_data[0]['tokens']}")
print(f"Tokenized Output IDs: {tokenized_output['input_ids']}")
print(f"Tokenized Output Tokens: {[tokenizer.vocab[idx - 1] if idx - 1 < len(tokenizer.vocab) else '[UNK]' for idx in tokenized_output['input_ids']]}")


Original Tokens: ['1.', 'Duloxetine', '60', 'mg', 'Capsule', 'Sig:', 'One', '(1)', 'Capsule', 'PO', 'DAILY', 'for', 'depression.']
Tokenized Output IDs: [59, 470, 153, 1653, 339, 1068, 897, 1, 339, 912, 413, 1511, 1415]
Tokenized Output Tokens: ['1.', 'Duloxetine', '60', 'mg', 'Capsule', 'Sig:', 'One', '(1)', 'Capsule', 'PO', 'DAILY', 'for', 'depression.']


In [13]:
# Custom padding function
def custom_pad_and_truncate(batch, max_length=48):
    """Pad and truncate the input IDs, attention masks, and labels."""
    padded_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for item in batch:
        # Pad or truncate input_ids
        input_ids = item['input_ids']
        input_ids = input_ids[:max_length] + [0] * max(0, max_length - len(input_ids))  # Pad with 0
        padded_batch["input_ids"].append(input_ids)

        # Pad or truncate attention_mask
        attention_mask = item['attention_mask']
        attention_mask = attention_mask[:max_length] + [0] * max(0, max_length - len(attention_mask))  # Pad with 0
        padded_batch["attention_mask"].append(attention_mask)

        # Pad or truncate labels
        labels = item['labels']
        labels = labels[:max_length] + [-100] * max(0, max_length - len(labels))  # Pad with -100 for labels
        padded_batch["labels"].append(labels)

    # Convert lists to PyTorch tensors
    padded_batch["input_ids"] = torch.tensor(padded_batch["input_ids"], dtype=torch.long)
    padded_batch["attention_mask"] = torch.tensor(padded_batch["attention_mask"], dtype=torch.long)
    padded_batch["labels"] = torch.tensor(padded_batch["labels"], dtype=torch.long)

    return padded_batch

# Custom data collator
def custom_data_collator(batch):
    """Custom data collator for batching and padding."""
    return custom_pad_and_truncate(batch, max_length=48)

# Metrics calculation function remains the same
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Extract true predictions and labels, ignoring the padding (-100)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists of true predictions and labels
    true_predictions_flat = [item for sublist in true_predictions for item in sublist]
    true_labels_flat = [item for sublist in true_labels for item in sublist]

    # Set zero_division to 0 to handle undefined precision/recall
    report = classification_report(true_labels_flat, true_predictions_flat, output_dict=True, zero_division=0)
    return {
        "precision": report["macro avg"]["precision"],
        "recall": report["macro avg"]["recall"],
        "f1": report["macro avg"]["f1-score"],
        "accuracy": report["accuracy"],
    }

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Ensure all inputs are on the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Debug print for the first training step (ensure minimal operations)
        if self.state.global_step == 0:  # Print only during the first step
            print("\n[DEBUG] Trainer Input:")
            for key, value in inputs.items():
                print(f"{key}: {value.shape}, dtype: {value.dtype}")

                # Use `detach` and `cpu` to safely access the data for printing without affecting training
                detached_value = value.detach().cpu()
                for i in range(min(3, detached_value.size(0))):  # Limit to the first 3 inputs
                    print(f"Sample {i + 1} of {key}: {detached_value[i][:10].tolist()}")  # Convert to list for clearer print

        # Forward pass through the model
        outputs = model(**inputs)
        logits = outputs.logits
        labels = inputs['labels']

        # Compute loss
        loss = self.label_smoother(outputs, labels) if self.label_smoother else outputs.loss

        return (loss, outputs) if return_outputs else loss

In [14]:
# Load model and move to the correct device
model = AutoModelForTokenClassification.from_pretrained("Lianglab/PharmBERT-uncased", num_labels=len(label_list))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Split the data
train_data, eval_data = train_test_split(updated_data, test_size=0.2,random_state=42)

# Create DataLoaders for training and evaluation using the custom data collator
train_dataloader = DataLoader(train_data, batch_size=16, collate_fn=custom_data_collator)
eval_dataloader = DataLoader(eval_data, batch_size=16, collate_fn=custom_data_collator)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,  # Increased learning rate
    per_device_train_batch_size=8,  # Reduced batch size
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to='none'
)

# Training and model setup remains the same
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,  # Use the Dataset objects here
    eval_dataset=eval_data,
    compute_metrics=compute_metrics,
    data_collator=custom_data_collator,
    tokenizer=tokenizer,  # Still using `tokenizer` for consistency
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Lianglab/PharmBERT-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Check the shapes and keys of the first batch from the train_dataloader
for batch in train_dataloader:
    # Move batch tensors to the appropriate device
    batch = {k: v.to(device) for k, v in batch.items()}

    # Print the keys in the batch dictionary
    print(f"Batch keys: {batch.keys()}")  # Should show the keys present in the batch (e.g., input_ids, attention_mask, labels)

    # Print the shape of each tensor in the batch
    for key in batch:
        print(f"{key} shape: {batch[key].shape}, dtype: {batch[key].dtype}")

    # Display a single entry from the batch for validation
    print("\n--- Sample Entry from First Batch ---")
    print("Input IDs:", batch['input_ids'][0].tolist())
    print("Tokens:", [new_tokens[id] if id < len(new_tokens) and id != 0 else "[PAD]" if id == 0 else "[UNK]" for id in batch['input_ids'][0].tolist()])
    print("Labels:", batch['labels'][0].tolist())
    print("Attention Mask:", batch['attention_mask'][0].tolist())

    # Break after inspecting the first batch
    break

Batch keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
input_ids shape: torch.Size([16, 48]), dtype: torch.int64
attention_mask shape: torch.Size([16, 48]), dtype: torch.int64
labels shape: torch.Size([16, 48]), dtype: torch.int64

--- Sample Entry from First Batch ---
Input IDs: [70, 801, 144, 1653, 1130, 1068, 897, 1, 1130, 912, 272, 1511, 1565, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokens: ['110', 'Metoprolol Succinate', '50,000', 'mg-300', 'Tablet,', 'Sildenafil', 'Onychomycosis', '(10)', 'Tablet,', 'PRN', 'BPH', 'formation', 'hyperthyroidism.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Labels: [0, 1, 3, 4, 5, 0, 7, 7, 5, 11, 9, 0, 13, -1

In [16]:
# Train and save the model
trainer.train()
trainer.save_model("./split_finetuned_pharmbert_model")


[DEBUG] Trainer Input:
input_ids: torch.Size([8, 48]), dtype: torch.int64
Sample 1 of input_ids: [158, 324, 84, 1653, 1130, 1068, 897, 1, 1130, 912]
Sample 2 of input_ids: [117, 1043, 64, 1653, 1130, 1068, 897, 1, 1130, 912]
Sample 3 of input_ids: [131, 801, 144, 1653, 1130, 1068, 897, 1, 1130, 912]
attention_mask: torch.Size([8, 48]), dtype: torch.int64
Sample 1 of attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 2 of attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 3 of attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
labels: torch.Size([8, 48]), dtype: torch.int64
Sample 1 of labels: [0, 1, 3, 4, 5, 0, 7, 7, 5, 11]
Sample 2 of labels: [0, 1, 3, 4, 5, 0, 7, 7, 5, 11]
Sample 3 of labels: [0, 1, 3, 4, 5, 0, 7, 7, 5, 11]


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.032800,0.026759,0.988846,0.992355,0.990573,0.992441
2,0.018000,0.021691,0.992344,0.994512,0.993404,0.995049
3,0.013300,0.015940,0.994206,0.995949,0.995062,0.995943
4,0.008900,0.015288,0.994825,0.996812,0.995810,0.996667
5,0.007100,0.014676,0.995144,0.996701,0.995916,0.996812


Tokenizer vocabulary saved to ./results/checkpoint-1503/vocab.json
Tokenizer vocabulary saved to ./results/checkpoint-3006/vocab.json
Tokenizer vocabulary saved to ./results/checkpoint-4509/vocab.json
Tokenizer vocabulary saved to ./results/checkpoint-6012/vocab.json
Tokenizer vocabulary saved to ./results/checkpoint-7515/vocab.json
Tokenizer vocabulary saved to ./results/checkpoint-7515/vocab.json
Tokenizer vocabulary saved to ./split_finetuned_pharmbert_model/vocab.json


# **EXAMPLE TEST**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install datasets

%cd /content/drive/Othercomputers/My MacBook Air/Thesis/Coding/Synth

Mounted at /content/drive
/content/drive/Othercomputers/My MacBook Air/Thesis/Coding/Synth


In [21]:
# Initialize a set to collect unique words from the data
new_tokens_set = set()

# Simple space-based tokenizer function for vocabulary extraction
def simple_tokenizer(text: str):
    return text.split()

# Iterate through each entry in the aligned_data
for entry in aligned_data:
    # Extract the instructions and split into words
    instruction_tokens = simple_tokenizer(entry['instructions'])
    new_tokens_set.update(instruction_tokens)  # Add tokens to the set

    # Extract medication details
    medications = entry['medications']

    # Iterate through each medication in the entry
    for medication_name, fields in medications.items():
        # Add the medication name itself
        new_tokens_set.add(medication_name)

        # Iterate through each field in the medication dictionary
        for field_name, field_value in fields.items():
            # Split the field value into tokens and add them
            field_tokens = simple_tokenizer(field_value)
            new_tokens_set.update(field_tokens)

# Convert the set to a sorted list to maintain a consistent order
new_tokens = sorted(list(new_tokens_set))


In [24]:
import os
import json
import torch
import numpy as np

# Define label list
label_list = ["O", "B-DRUG", "I-DRUG", "B-STRENGTH", "I-STRENGTH",
              "B-FORM", "I-FORM", "B-DOSAGE", "I-DOSAGE",
              "B-FREQUENCY", "I-FREQUENCY", "B-ROUTE", "I-ROUTE",
              "B-REASON", "I-REASON"]

# Check if a GPU is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your custom tokenizer class
class SimpleTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.token_to_id = {token: idx + 1 for idx, token in enumerate(vocab)}  # ID starts at 1
        self.unknown_token_id = len(self.token_to_id) + 1  # ID for unknown tokens

    def tokenize(self, text: str) -> list:
        return text.split()  # Simple space-based tokenization

    def convert_tokens_to_ids(self, tokens: list) -> list:
        return [self.token_to_id.get(token, self.unknown_token_id) for token in tokens]

    def __call__(self, text: str, **kwargs) -> dict:
        tokens = self.tokenize(text)
        input_ids = self.convert_tokens_to_ids(tokens)
        return {
            "input_ids": input_ids,
            "attention_mask": [1] * len(input_ids)  # Default attention mask for each token
        }

# Initialize your SimpleTokenizer with your vocabulary, new_tokens defined above
tokenizer = SimpleTokenizer(vocab=new_tokens)

# Function to predict and compare labels
def predict_and_evaluate(tokens, ground_truth_labels, model, tokenizer, label_list, max_length=48):
    # Tokenize input using your SimpleTokenizer
    inputs = tokenizer(' '.join(tokens))  # Join tokens into a single string for tokenization

    # Pad or truncate input IDs and attention masks
    input_ids = inputs['input_ids'][:max_length] + [0] * max(0, max_length - len(inputs['input_ids']))
    attention_mask = inputs['attention_mask'][:max_length] + [0] * max(0, max_length - len(inputs['attention_mask']))

    # Convert to tensors and move to device
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Add batch dimension and move to device
    attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)

    # Get model predictions
    model.eval()  # Put the model in evaluation mode
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=2).cpu().numpy()

    # Convert predicted label ids to actual label names
    predicted_labels = [label_list[p] for p in predictions[0]]

    return tokens, predicted_labels  # Return tokens and predicted labels

# Initialize accuracy counters
correct_preds = {label: 0 for label in label_list}
total_preds = {label: 0 for label in label_list}

# Read the JSON files and evaluate the predictions
json_folder = "test/labeled_records"
all_records = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Flag to control sample entry printing
printed_sample = False
record_count = 0

for record_file in all_records:
    # Print statement for every 20 records processed
    if record_count % 20 == 0:
        print(f"Processed 20 records")  # Print statement to show record processing

    with open(os.path.join(json_folder, record_file), 'r') as json_file:
        data = json.load(json_file)
        entries = data['entries']

        for entry in entries:
            tokens = entry['tokens']
            ground_truth_labels = entry['labels']

            # Get model predictions
            predicted_tokens, predicted_labels = predict_and_evaluate(
                tokens, ground_truth_labels, model, tokenizer, label_list
            )

            # Update accuracy counts
            for pred_label, true_label in zip(predicted_labels, ground_truth_labels):
                if true_label in label_list:  # Ensure it's a valid label to evaluate
                    total_preds[true_label] += 1
                    if pred_label == true_label:
                        correct_preds[true_label] += 1

            # Show sample entry only for the first record
            if not printed_sample:
                print(f"\n--- Sample Entry from Record 1 ---")
                print(f"Tokens: {tokens}")
                print(f"Ground Truth Labels: {ground_truth_labels}")
                print(f"Predicted Labels  : {predicted_labels}")
                printed_sample = True  # Set flag to True after printing the first sample

    record_count += 1  # Increment the record count after processing each file

# Calculate overall accuracy for each label
overall_accuracies = {label: (correct_preds[label] / total_preds[label]) if total_preds[label] > 0 else 0 for label in label_list}

# Display the overall accuracy for each field
print("\nOverall Accuracy per Field:")
for label, accuracy in overall_accuracies.items():
    print(f"{label:15}: {accuracy:.2f}")


Processed 20 records

--- Sample Entry from Record 1 ---
Tokens: ['1.', 'Fluticasone-Salmeterol', '250-50', 'mcg/Dose', 'Disk', 'with', 'Device', 'Sig:', 'One', '(1)', 'Disk', 'with', 'Device', 'Inhalation', '[**Hospital1', '**]', '(2', 'times', 'a', 'day).', 'Disp:*60', 'Disk', 'with', 'Device(s)*', 'Refills:*2*']
Ground Truth Labels: ['O', 'B-DRUG', 'B-STRENGTH', 'I-STRENGTH', 'B-FORM', 'O', 'B-FORM', 'O', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'O', 'I-FORM', 'B-ROUTE', 'O', 'O', 'B-DOSAGE', 'I-DOSAGE', 'I-DOSAGE', 'B-FREQUENCY', 'O', 'B-FORM', 'O', 'B-FORM', 'O']
Predicted Labels  : ['O', 'B-DRUG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DOSAGE', 'B-DOSAGE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DOSAGE', 'O', 'O', 'O', 'O', 'B-DOSAGE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DOSAGE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DOSAGE', 'B-DOSAGE', 'B-DOSAGE', 'O']
Processed 20 records
Processed 20 records
Processed 20 records
Processed 20 records

Overall Accuracy per Fiel

In [25]:
import torch
import numpy as np
from transformers import AutoModelForTokenClassification

# Define label list
label_list = ["O", "B-DRUG", "I-DRUG", "B-STRENGTH", "I-STRENGTH",
              "B-FORM", "I-FORM", "B-DOSAGE", "I-DOSAGE",
              "B-FREQUENCY", "I-FREQUENCY", "B-ROUTE", "I-ROUTE",
              "B-REASON", "I-REASON"]

# Load model
model = AutoModelForTokenClassification.from_pretrained("./split_finetuned_pharmbert_model", num_labels=len(label_list))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define your custom SimpleTokenizer
class SimpleTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.token_to_id = {token: idx + 1 for idx, token in enumerate(vocab)}  # ID starts at 1
        self.unknown_token_id = len(self.token_to_id) + 1  # ID for unknown tokens

    def tokenize(self, text: str) -> list:
        return text.split()  # Simple space-based tokenization

    def convert_tokens_to_ids(self, tokens: list) -> list:
        return [self.token_to_id.get(token, self.unknown_token_id) for token in tokens]

    def __call__(self, text: str):
        tokens = self.tokenize(text)
        input_ids = self.convert_tokens_to_ids(tokens)
        return {
            "input_ids": input_ids,
            "attention_mask": [1] * len(input_ids)  # Default attention mask for each token
        }

# Initialize your tokenizer with the new vocabulary
# Assuming 'new_tokens' is the list of unique tokens you've created previously
tokenizer = SimpleTokenizer(vocab=new_tokens)

# Function to predict labels using the SimpleTokenizer
def predict_single_example_untokenized(input_example, model, tokenizer, label_list, max_length=48):
    # Use simple space-based tokenizer
    simple_tokens = tokenizer.tokenize(input_example)

    # Prepare input tensor
    inputs = tokenizer(input_example)

    # Convert input_ids to tensor
    input_ids = torch.tensor(inputs["input_ids"]).unsqueeze(0).to(device)  # Add batch dimension and move to device
    attention_mask = torch.tensor(inputs["attention_mask"]).unsqueeze(0).to(device)  # Add batch dimension

    # Put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=2).cpu().numpy()

    # Convert predicted label ids to actual label names
    predicted_labels = [label_list[p] for p in predictions[0]]

    # Align the length of predicted labels with simple tokens
    predicted_labels = predicted_labels[:len(simple_tokens)]

    # Print the results
    print("\n--- Prediction for Single Example ---")
    for token, label in zip(simple_tokens, predicted_labels):
        print(f"{token}: {label}")

# Manually provide an untokenized example input
example_input = "2. Rosuvastatin 10 mg Tablet Sig: One (1) Tablet PO QHS for high cholesterol."

# Run the prediction
predict_single_example_untokenized(example_input, model, tokenizer, label_list)



--- Prediction for Single Example ---
2.: O
Rosuvastatin: B-DRUG
10: B-STRENGTH
mg: I-STRENGTH
Tablet: B-FORM
Sig:: O
One: B-DOSAGE
(1): B-DOSAGE
Tablet: B-FORM
PO: B-ROUTE
QHS: B-FREQUENCY
for: O
high: B-REASON
cholesterol.: B-REASON
